In [1]:
with open("inputs/Day_07.txt") as f:
    raw_puzzle_data = f.read()

In [9]:
import itertools

def part_1_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    max_signal = 0
    for config in itertools.permutations(range(5)):
        prev_output = 0
        for amp_idx in range(5):
            computer_inputs = (config[amp_idx], prev_output)
            _, prev_output = computer(puzzle_input.copy(), computer_inputs)
                
        max_signal = max(prev_output, max_signal)
        
    return max_signal


def computer(sequence, inputs):
    index = 0
    input_idx = 0
    diag_nbr = None
    while True:
        opt_code = sequence[index]
        opt_code_with_modes = str(opt_code).zfill(4)
        opt_code = int(opt_code_with_modes[-2:])
        
        try:
            par_1 = sequence[sequence[index + 1]] if opt_code_with_modes[1] == "0" else sequence[index + 1]
            par_2 = sequence[sequence[index + 2]] if opt_code_with_modes[0] == "0" else sequence[index + 2]
        except:
            pass
        
        if opt_code == 99:
            return sequence, diag_nbr
        elif opt_code == 3:
            # input
            target_address = sequence[index + 1]
            sequence[target_address] = inputs[input_idx]
            input_idx += 1
            index += 2
        elif opt_code == 4:
            # output
            diag_nbr = par_1
            index += 2
        elif opt_code == 1:
            target_address = sequence[index + 3]
            sequence[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = sequence[index + 3]
            sequence[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = sequence[index + 3]
            sequence[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = sequence[index + 3]
            sequence[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        else:
            print(f"Wrong code: {opt_code}")

In [10]:
print(f"Part 1 solution: {part_1_solution(raw_puzzle_data)}")

Part 1 solution: 212460


In [223]:
import itertools

def part_2_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    max_signal = 0
#     possible_phaze_settings = itertools.permutations(range(5, 10))
    for phaze_settings in itertools.permutations(range(5, 10)):
        # setup computers
        system = list()
        for amp_idx in range(5):
            amp = computer(puzzle_input.copy(), phaze_settings[amp_idx])
            #process until first input is reached
            next(amp)
            system.append(amp)
            
        signal = run_amp_system(system)
                
        max_signal = max(signal, max_signal)
        
    return max_signal


def run_amp_system(system):
    signal = 0
    while True:
        for amplifier in system:
            try:
#                 signal = amplifier.send(signal)
                amplifier.send(signal)
                signal = next(amplifier)
            except StopIteration:
                return signal


def computer(sequence, phaze_setting):
    index = 0
    input_idx = 0
    diag_nbr = None
    opt_code = -1 # start
    while True:
        opt_code = sequence[index]
        opt_code_with_modes = str(opt_code).zfill(4)
        opt_code = int(opt_code_with_modes[-2:])
        
        try:
            par_1 = sequence[sequence[index + 1]] if opt_code_with_modes[1] == "0" else sequence[index + 1]
            par_2 = sequence[sequence[index + 2]] if opt_code_with_modes[0] == "0" else sequence[index + 2]
        except:
            pass
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            # input
#             print(f"in {input_idx}")
            target_address = sequence[index + 1]
            if input_idx == 0:
                sequence[target_address] = phaze_setting
            else:
                sequence[target_address] = yield diag_nbr
            input_idx += 1
            index += 2
        elif opt_code == 4:
            # output
#             print("out")
            diag_nbr = par_1
            yield diag_nbr
            index += 2
        elif opt_code == 1:
            target_address = sequence[index + 3]
            sequence[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = sequence[index + 3]
            sequence[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = sequence[index + 3]
            sequence[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = sequence[index + 3]
            sequence[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        else:
            print(f"Wrong code: {opt_code}")
                    
    yield diag_nbr

In [224]:
raw_test_input = """3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"""
assert(part_2_solution(raw_test_input) == 139629729)
raw_test_input = """3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"""
assert(part_2_solution(raw_test_input) == 18216)
print("Tests passed")

Tests passed


In [225]:
print(f"Part 2 solution: {part_2_solution(raw_puzzle_data)}")

Part 2 solution: 21844737


In [226]:
%%timeit
part_2_solution(raw_puzzle_data)

48.8 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
